# Imports

In [112]:
import sqlite3
import pandas as pd
import numpy as np
import pickle
import seaborn as sns

# Label Encoder
from sklearn.preprocessing import LabelEncoder
# One Hot Encoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
# Escalonamento - Padronização
from sklearn.preprocessing import StandardScaler
# Dividir base de dados em treino e teste
from sklearn.model_selection import train_test_split
# Naive Bayes
from sklearn.naive_bayes import GaussianNB
# Acurácia do algoritmo
from sklearn.metrics import accuracy_score
# Matriz de confusão
from sklearn.metrics import confusion_matrix
# Relatório
from sklearn.metrics import classification_report
# Matriz de confusão
from yellowbrick.classifier import ConfusionMatrix

# Loading data

In [113]:
data = '/home/uadson/repos/machine-learn/data/database/chatlocaldb.sqlite3'

In [114]:
conn = sqlite3.connect(data)
services_qs = "SELECT * FROM services_service;"
questions_qs = "SELECT * FROM questions_question;"
services_df = pd.read_sql_query(services_qs, conn)
questions_df = pd.read_sql_query(questions_qs, conn)

In [115]:
services_df.head(5)

,id,created_at,updated_at,active,name,index,department_id,profile_id
0,1,2023-09-15 15:04:26.549000,2024-01-16 13:26:17.796000,1,ENTENDA SEU IPTU,1,1,4
1,2,2023-09-15 15:14:38.771000,2023-09-15 15:14:38.775000,1,NOTA FISCAL,2,1,5
2,3,2023-09-15 15:32:25.555000,2024-01-24 10:06:59.069000,1,SIMPLES NACIONAL 2024,3,1,5
3,4,2023-09-15 15:42:50.880000,2023-09-18 13:36:32.251000,1,CADASTRO MOBILIÁRIO 1,4,1,5
4,5,2023-09-15 15:47:02.408000,2023-09-15 16:12:51.991000,1,ISS,5,1,5


In [116]:
questions_df.head(5)

,id,created_at,updated_at,active,title,answer,url,index,department_id,profile_id,service_id
0,1,2023-09-15 15:09:14.343000,2024-01-16 15:33:35.386000,1,Como emitir o boleto do IPTU 2024?,O procedimento é simples. Após acessar o porta...,https://www.goiania.go.gov.br/sefin/iptu/,1,1,4,1
1,2,2023-09-15 15:11:04.308000,2023-12-05 11:04:00.894000,1,Onde encontro o número de inscrição do meu imó...,Em qualquer boleto de IPTU dos anos anteriores...,None,2,1,4,1
2,3,2023-09-15 15:12:06.972000,2024-01-16 14:00:26.230000,1,Qual será o reajuste do IPTU em 2024?,"Até o ano de 2025, o IPTU não terá reajuste do...",None,3,1,4,1
3,4,2023-09-15 15:12:54.616000,2024-01-16 14:02:04.196000,1,Quem tem direito à isenção do IPTU 2024?,"Em Goiânia, o Código Tributário Municipal busc...",None,4,1,4,1
4,5,2023-09-15 15:15:50.990000,2023-12-04 13:17:34.529000,1,Como faço para cancelar a Nota Fiscal?,O próprio contribuinte pode efetuar o cancelam...,None,1,1,5,2


# Preprocessing

In [117]:
services = services_df['id name'.split(' ')]
services.head()

,id,name
0,1,ENTENDA SEU IPTU
1,2,NOTA FISCAL
2,3,SIMPLES NACIONAL 2024
3,4,CADASTRO MOBILIÁRIO 1
4,5,ISS


In [118]:
questions = questions_df['title answer url department_id service_id'.split(' ')]
questions.head(5)

,title,answer,url,department_id,service_id
0,Como emitir o boleto do IPTU 2024?,O procedimento é simples. Após acessar o porta...,https://www.goiania.go.gov.br/sefin/iptu/,1,1
1,Onde encontro o número de inscrição do meu imó...,Em qualquer boleto de IPTU dos anos anteriores...,None,1,1
2,Qual será o reajuste do IPTU em 2024?,"Até o ano de 2025, o IPTU não terá reajuste do...",None,1,1
3,Quem tem direito à isenção do IPTU 2024?,"Em Goiânia, o Código Tributário Municipal busc...",None,1,1
4,Como faço para cancelar a Nota Fiscal?,O próprio contribuinte pode efetuar o cancelam...,None,1,2


In [119]:
# Alterando o nome das colunas
questions.columns = ['title answer url department service'.split(' ')]

In [120]:
questions.head(5)

,title,answer,url,department,service
0,Como emitir o boleto do IPTU 2024?,O procedimento é simples. Após acessar o porta...,https://www.goiania.go.gov.br/sefin/iptu/,1,1
1,Onde encontro o número de inscrição do meu imó...,Em qualquer boleto de IPTU dos anos anteriores...,None,1,1
2,Qual será o reajuste do IPTU em 2024?,"Até o ano de 2025, o IPTU não terá reajuste do...",None,1,1
3,Quem tem direito à isenção do IPTU 2024?,"Em Goiânia, o Código Tributário Municipal busc...",None,1,1
4,Como faço para cancelar a Nota Fiscal?,O próprio contribuinte pode efetuar o cancelam...,None,1,2


In [121]:
questions['department'] = questions['department'].replace(1, 'Secretaria Municipal de Finanças');

/tmp/ipykernel_236877/3229214158.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  questions['department'] = questions['department'].replace(1, 'Secretaria Municipal de Finanças');


In [122]:
questions.head()

,title,answer,url,department,service
0,Como emitir o boleto do IPTU 2024?,O procedimento é simples. Após acessar o porta...,https://www.goiania.go.gov.br/sefin/iptu/,Secretaria Municipal de Finanças,1
1,Onde encontro o número de inscrição do meu imó...,Em qualquer boleto de IPTU dos anos anteriores...,None,Secretaria Municipal de Finanças,1
2,Qual será o reajuste do IPTU em 2024?,"Até o ano de 2025, o IPTU não terá reajuste do...",None,Secretaria Municipal de Finanças,1
3,Quem tem direito à isenção do IPTU 2024?,"Em Goiânia, o Código Tributário Municipal busc...",None,Secretaria Municipal de Finanças,1
4,Como faço para cancelar a Nota Fiscal?,O próprio contribuinte pode efetuar o cancelam...,None,Secretaria Municipal de Finanças,2


In [123]:
for qcol in range(len(questions['service'])):
    pk_value = questions.iloc[qcol, 4]
    for scol in range(len(services['id'])):
        id_value = services.iloc[scol, 0]
        #print(pk_value, id_value)
        if pk_value == id_value:
            # print(pk_value, services.iloc[scol, 1])
            questions['service'] = questions['service'].replace(pk_value, services.iloc[scol, 1])

/tmp/ipykernel_236877/2958823446.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  questions['service'] = questions['service'].replace(pk_value, services.iloc[scol, 1])


In [124]:
questions.head()

,title,answer,url,department,service
0,Como emitir o boleto do IPTU 2024?,O procedimento é simples. Após acessar o porta...,https://www.goiania.go.gov.br/sefin/iptu/,Secretaria Municipal de Finanças,ENTENDA SEU IPTU
1,Onde encontro o número de inscrição do meu imó...,Em qualquer boleto de IPTU dos anos anteriores...,None,Secretaria Municipal de Finanças,ENTENDA SEU IPTU
2,Qual será o reajuste do IPTU em 2024?,"Até o ano de 2025, o IPTU não terá reajuste do...",None,Secretaria Municipal de Finanças,ENTENDA SEU IPTU
3,Quem tem direito à isenção do IPTU 2024?,"Em Goiânia, o Código Tributário Municipal busc...",None,Secretaria Municipal de Finanças,ENTENDA SEU IPTU
4,Como faço para cancelar a Nota Fiscal?,O próprio contribuinte pode efetuar o cancelam...,None,Secretaria Municipal de Finanças,NOTA FISCAL


In [125]:
questions['url'] = questions['url'].replace('', '-')

/tmp/ipykernel_236877/2762199738.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  questions['url'] = questions['url'].replace('', '-')


In [126]:
questions

,title,answer,url,department,service
0,Como emitir o boleto do IPTU 2024?,O procedimento é simples. Após acessar o porta...,https://www.goiania.go.gov.br/sefin/iptu/,Secretaria Municipal de Finanças,ENTENDA SEU IPTU
1,Onde encontro o número de inscrição do meu imó...,Em qualquer boleto de IPTU dos anos anteriores...,None,Secretaria Municipal de Finanças,ENTENDA SEU IPTU
2,Qual será o reajuste do IPTU em 2024?,"Até o ano de 2025, o IPTU não terá reajuste do...",None,Secretaria Municipal de Finanças,ENTENDA SEU IPTU
3,Quem tem direito à isenção do IPTU 2024?,"Em Goiânia, o Código Tributário Municipal busc...",None,Secretaria Municipal de Finanças,ENTENDA SEU IPTU
4,Como faço para cancelar a Nota Fiscal?,O próprio contribuinte pode efetuar o cancelam...,None,Secretaria Municipal de Finanças,NOTA FISCAL
...,...,...,...,...,...
129,Quem deve solicitar opção pelo Simples Nacional?,Os contribuintes que foram excluídos por qualq...,-,Secretaria Municipal de Finanças,SIMPLES NACIONAL 2024
130,Qual o prazo para solicitar opção?,A opção deverá ser formalizada até o último di...,-,Secretaria Municipal de Finanças,SIMPLES NACIONAL 2024
131,"Uma vez solicitada, a opção pelo Simples Nacio...",Enquanto não vencido o prazo para formalização...,-,Secretaria Municipal de Finanças,SIMPLES NACIONAL 2024
132,"Após cancelada, poderá ser formalizada nova op...","Após cancelado o pedido, poderá ser formalizad...",-,Secretaria Municipal de Finanças,SIMPLES NACIONAL 2024


In [127]:
questions.isnull()

,title,answer,url,department,service
0,False,False,False,False,False
1,False,False,True,False,False
2,False,False,True,False,False
3,False,False,True,False,False
4,False,False,True,False,False
...,...,...,...,...,...
129,False,False,False,False,False
130,False,False,False,False,False
131,False,False,False,False,False
132,False,False,False,False,False


In [128]:
questions = questions.fillna('-')
questions

,title,answer,url,department,service
0,Como emitir o boleto do IPTU 2024?,O procedimento é simples. Após acessar o porta...,https://www.goiania.go.gov.br/sefin/iptu/,Secretaria Municipal de Finanças,ENTENDA SEU IPTU
1,Onde encontro o número de inscrição do meu imó...,Em qualquer boleto de IPTU dos anos anteriores...,-,Secretaria Municipal de Finanças,ENTENDA SEU IPTU
2,Qual será o reajuste do IPTU em 2024?,"Até o ano de 2025, o IPTU não terá reajuste do...",-,Secretaria Municipal de Finanças,ENTENDA SEU IPTU
3,Quem tem direito à isenção do IPTU 2024?,"Em Goiânia, o Código Tributário Municipal busc...",-,Secretaria Municipal de Finanças,ENTENDA SEU IPTU
4,Como faço para cancelar a Nota Fiscal?,O próprio contribuinte pode efetuar o cancelam...,-,Secretaria Municipal de Finanças,NOTA FISCAL
...,...,...,...,...,...
129,Quem deve solicitar opção pelo Simples Nacional?,Os contribuintes que foram excluídos por qualq...,-,Secretaria Municipal de Finanças,SIMPLES NACIONAL 2024
130,Qual o prazo para solicitar opção?,A opção deverá ser formalizada até o último di...,-,Secretaria Municipal de Finanças,SIMPLES NACIONAL 2024
131,"Uma vez solicitada, a opção pelo Simples Nacio...",Enquanto não vencido o prazo para formalização...,-,Secretaria Municipal de Finanças,SIMPLES NACIONAL 2024
132,"Após cancelada, poderá ser formalizada nova op...","Após cancelado o pedido, poderá ser formalizad...",-,Secretaria Municipal de Finanças,SIMPLES NACIONAL 2024


In [129]:
questions_data = questions.iloc[:, [0, 3, 4, 1]]
questions_data

,title,department,service,answer
0,Como emitir o boleto do IPTU 2024?,Secretaria Municipal de Finanças,ENTENDA SEU IPTU,O procedimento é simples. Após acessar o porta...
1,Onde encontro o número de inscrição do meu imó...,Secretaria Municipal de Finanças,ENTENDA SEU IPTU,Em qualquer boleto de IPTU dos anos anteriores...
2,Qual será o reajuste do IPTU em 2024?,Secretaria Municipal de Finanças,ENTENDA SEU IPTU,"Até o ano de 2025, o IPTU não terá reajuste do..."
3,Quem tem direito à isenção do IPTU 2024?,Secretaria Municipal de Finanças,ENTENDA SEU IPTU,"Em Goiânia, o Código Tributário Municipal busc..."
4,Como faço para cancelar a Nota Fiscal?,Secretaria Municipal de Finanças,NOTA FISCAL,O próprio contribuinte pode efetuar o cancelam...
...,...,...,...,...
129,Quem deve solicitar opção pelo Simples Nacional?,Secretaria Municipal de Finanças,SIMPLES NACIONAL 2024,Os contribuintes que foram excluídos por qualq...
130,Qual o prazo para solicitar opção?,Secretaria Municipal de Finanças,SIMPLES NACIONAL 2024,A opção deverá ser formalizada até o último di...
131,"Uma vez solicitada, a opção pelo Simples Nacio...",Secretaria Municipal de Finanças,SIMPLES NACIONAL 2024,Enquanto não vencido o prazo para formalização...
132,"Após cancelada, poderá ser formalizada nova op...",Secretaria Municipal de Finanças,SIMPLES NACIONAL 2024,"Após cancelado o pedido, poderá ser formalizad..."


In [130]:
X_questions = questions_data.iloc[:, 0:3].values
X_questions[0]

array(['Como emitir o boleto do IPTU 2024?',
       'Secretaria Municipal de Finanças', 'ENTENDA SEU IPTU'],
      dtype=object)

In [131]:
y_questions = questions_data.iloc[:, 3].values
y_questions[0]

'O procedimento é simples. Após acessar o portal da Prefeitura, www.goiania.go.gov.br, o interessado tem que clicar no link “Débitos IPTU e ITU 2024”, na tela principal do site. Em seguida inserir o número da inscrição cadastral do proprietário do imóvel para visualizar o boleto e o valor a ser pago. A ferramenta ainda disponibiliza os dados cadastrais do imóvel para conferência dos valores.'

### Training

In [132]:
label_encoder_title = LabelEncoder().fit_transform(X_questions[:, 0])
label_encoder_department = LabelEncoder().fit_transform(X_questions[:, 1])
label_encoder_service = LabelEncoder().fit_transform(X_questions[:, 2])

In [133]:
X_questions[:, 0] = label_encoder_title
X_questions[:, 1] = label_encoder_department
X_questions[:, 2] = label_encoder_service

In [134]:
X_questions

array([[12, 0, 2],
       [65, 0, 2],
       [111, 0, 2],
       [123, 0, 2],
       [17, 0, 7],
       [26, 0, 2],
       [129, 0, 7],
       [2, 0, 2],
       [59, 0, 7],
       [94, 0, 2],
       [60, 0, 7],
       [131, 0, 7],
       [125, 0, 7],
       [45, 0, 7],
       [128, 0, 7],
       [42, 0, 7],
       [9, 0, 12],
       [85, 0, 12],
       [23, 0, 12],
       [22, 0, 12],
       [106, 0, 12],
       [1, 0, 12],
       [57, 0, 12],
       [58, 0, 12],
       [39, 0, 12],
       [126, 0, 12],
       [120, 0, 2],
       [95, 0, 12],
       [24, 0, 12],
       [3, 0, 12],
       [44, 0, 12],
       [29, 0, 0],
       [93, 0, 0],
       [69, 0, 5],
       [7, 0, 2],
       [51, 0, 2],
       [89, 0, 2],
       [11, 0, 2],
       [113, 0, 11],
       [72, 0, 11],
       [87, 0, 11],
       [88, 0, 2],
       [25, 0, 2],
       [13, 0, 1],
       [101, 0, 6],
       [37, 0, 6],
       [104, 0, 6],
       [91, 0, 6],
       [96, 0, 6],
       [118, 0, 6],
       [38, 0, 6],
      

In [135]:
len(np.unique(questions['department']))

1

In [136]:
onehotencoder_questions = ColumnTransformer(transformers=[('OneHot', OneHotEncoder(), [0, 1, 2])], remainder='passthrough', )

In [137]:
X_questions = onehotencoder_questions.fit_transform(X_questions).toarray()

In [138]:
len(X_questions)

134

In [151]:
X_questions[0]

array([-0.086711  , -0.086711  , -0.086711  , -0.086711  , -0.086711  ,
       -0.086711  , -0.086711  , -0.086711  , -0.086711  , -0.086711  ,
       -0.086711  , -0.086711  , 11.53256259, -0.086711  , -0.086711  ,
       -0.086711  , -0.086711  , -0.086711  , -0.086711  , -0.086711  ,
       -0.086711  , -0.086711  , -0.086711  , -0.086711  , -0.086711  ,
       -0.086711  , -0.086711  , -0.086711  , -0.086711  , -0.086711  ,
       -0.086711  , -0.086711  , -0.086711  , -0.086711  , -0.086711  ,
       -0.086711  , -0.086711  , -0.086711  , -0.086711  , -0.086711  ,
       -0.086711  , -0.086711  , -0.086711  , -0.086711  , -0.086711  ,
       -0.086711  , -0.086711  , -0.086711  , -0.086711  , -0.086711  ,
       -0.086711  , -0.086711  , -0.086711  , -0.086711  , -0.086711  ,
       -0.086711  , -0.086711  , -0.086711  , -0.086711  , -0.086711  ,
       -0.086711  , -0.086711  , -0.086711  , -0.086711  , -0.086711  ,
       -0.086711  , -0.086711  , -0.086711  , -0.086711  , -0.08

In [140]:
X_questions.shape

(134, 150)

In [153]:
scaler_questions = StandardScaler()

X_questions = scaler_questions.fit_transform(X_questions)


In [155]:
X_questions[0]

array([-0.086711  , -0.086711  , -0.086711  , -0.086711  , -0.086711  ,
       -0.086711  , -0.086711  , -0.086711  , -0.086711  , -0.086711  ,
       -0.086711  , -0.086711  , 11.53256259, -0.086711  , -0.086711  ,
       -0.086711  , -0.086711  , -0.086711  , -0.086711  , -0.086711  ,
       -0.086711  , -0.086711  , -0.086711  , -0.086711  , -0.086711  ,
       -0.086711  , -0.086711  , -0.086711  , -0.086711  , -0.086711  ,
       -0.086711  , -0.086711  , -0.086711  , -0.086711  , -0.086711  ,
       -0.086711  , -0.086711  , -0.086711  , -0.086711  , -0.086711  ,
       -0.086711  , -0.086711  , -0.086711  , -0.086711  , -0.086711  ,
       -0.086711  , -0.086711  , -0.086711  , -0.086711  , -0.086711  ,
       -0.086711  , -0.086711  , -0.086711  , -0.086711  , -0.086711  ,
       -0.086711  , -0.086711  , -0.086711  , -0.086711  , -0.086711  ,
       -0.086711  , -0.086711  , -0.086711  , -0.086711  , -0.086711  ,
       -0.086711  , -0.086711  , -0.086711  , -0.086711  , -0.08

In [156]:
X_questions_train, X_questions_test, y_question_train, y_questions_test = train_test_split(X_questions, y_questions, test_size=0.30, random_state=0)

In [143]:
X_questions_train.shape, y_question_train.shape

((87, 150), (87,))

In [144]:
naive_questions_data = GaussianNB()
naive_questions_data.fit(X_questions_train, y_question_train)

GaussianNB()

In [145]:
predictions = naive_questions_data.predict(X_questions_test)
predictions

array(['As pendências (fiscal e cadastral) poderão ser consultados no Portal do Contribuinte, disponível no site da Prefeitura de Goiânia, www.goiania.go.gov.br, no menu destinado à verificação de pendências do Simples Nacional, ou por meio do link disponível\r\nno próprio Termo de Exclusão do Simples Nacional.',
       '1º quando o contribuinte não puder solicitar o cadastro eventual –CEV presencialmente em uma das agências do Atende Fácil, por agendamento pelo site www.goiania.go.gov.br, poderá encaminhar a solicitação por e-mail: gernot.sefin@goiania.go.gov.br, juntamente com os dados cadastrais:\r\n· Anexar documento pessoal (pessoa física) e endereço do responsável técnico pela obra ou se for pessoa jurídica anexar o cartão do CNPJ da empresa.\r\n· Anexar carteira do CREA ou CAU do responsável técnico.\r\n· Informar um e-mail para o cadastro.\r\n2º após o registro do cadastro eventual, em 24 horas será disparado uma senha de acesso do Portal do Contribuinte via e-mail cadastrado p

In [146]:
y_questions_test

array(['Todas as Microempresas (ME) e Empresas de Pequeno Porte (EPP) que em algum período do ano-calendário se encontravam como optantes pelo Simples Nacional. Também será permitida a entrega da DASN por empresas que não constam como optantes em algum período do ano-calendário, desde que possuam processo formalizado em uma das unidades das fazendas Federal, Estadual ou Municipal.',
       '1º- Quando o contribuinte não puder solicitar presencialmente em uma das agências do Atende Fácil, com agendamento pelo site www.goiania.go.gov.br, poderá encaminhar a solicitação por e-mail: gernot.sefin@goiania.go.gov.br, juntamente com os dados cadastrais, no qual receberá toda a orientação via e-mail.\r\n· Encaminhar cópia do Cartão do CNPJ da empresa.\r\n· Se necessário, a nota fiscal para verificar a incidência do ISS do serviço prestado.\r\n\r\n2º- Após a efetivação do Cadastro Eventual –CEV, o contribuinte poderá gerar a guia do ISS pelo link: \r\nhttps://www.goiania.go.gov.br/sistemas/saces

In [147]:
accuracy_score(y_questions_test, predictions)

0.0

In [148]:
confusion_matrix(y_questions_test, predictions)

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [149]:
cm = ConfusionMatrix(naive_questions_data)
cm.fit(X_questions_train, y_question_train)
cm.score(X_questions_test, y_questions_test)

ValueError: At least one label specified must be in y_true